In [ ]:
import pandas as pd
import numpy as np

In [ ]:
breast_cancer_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/bc_full_icd.csv')
breast_cancer_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/bc_full_snomed.csv')

crc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/crc_full_icd.csv')
crc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/crc_full_snomed.csv')

ec_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ec_full_icd.csv')
ec_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ec_full_snomed.csv')

ov_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ov_full_icd.csv')
ov_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/ov_full_snomed.csv')


pc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/pancreatic_full_icd.csv')
pc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/pancreatic_full_snomed.csv')


rc_icd_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/rc_full_icd.csv')
rc_snomed_df = pd.read_csv('/data/volume02/yuba/HC_STRUC/rc_full_snomed.csv')

In [ ]:
ov_snomed_df.dtypes

**DATABASE CONSIST OF 2 DIFFERENT DIAGNOSIS TABLES**
- Required to pass through function before merging for full diagnosis dataframe

In [ ]:
def unique_patients(icd_df, snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=None):

    icd_df_unique = icd_df.drop_duplicates(subset=[id_col, enc_col], keep='first').copy()
    snomed_df_unique = snomed_df.drop_duplicates(subset=[id_col, enc_col], keep='first').copy()
    
    #overlap logic
    overlap = pd.merge(
        icd_df_unique[[id_col, enc_col]],
        snomed_df_unique[[id_col, enc_col]],
        on=[id_col, enc_col],
        how='inner'
    )
    
    n_patients = overlap[id_col].nunique()
    print(f"{n_patients} unique patient(s) has overlapping encounters and will be dropped.")
    
    if not overlap.empty:
        snomed_df_unique = snomed_df_unique.merge(overlap, on=[id_col, enc_col], how='left', indicator=True)
        snomed_df_unique = snomed_df_unique[snomed_df_unique['_merge'] == 'left_only']
        snomed_df_unique = snomed_df_unique.drop(columns=['_merge'])
        
    merged_df = pd.concat([icd_df_unique, snomed_df_unique], ignore_index=True)
    
    if int_cols:
        for col in int_cols:
            if col in merged_df.columns:
                merged_df[col] = merged_df[col].astype('Int64')
    
    return merged_df
        
    #return icd_df_unique.reset_index(drop=True), snomed_df_unique.reset_index(drop=True)

In [ ]:
bc_df = unique_patients(breast_cancer_icd_df, breast_cancer_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

crc_df = unique_patients(crc_icd_df, crc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

ec_df = unique_patients(ec_icd_df, ec_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

ov_df = unique_patients(ov_icd_df, ov_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

pc_df = unique_patients(pc_icd_df, pc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])

rc_df = unique_patients(rc_icd_df, rc_snomed_df, id_col = 'PATIENT_IDENTIFIER', enc_col = 'ENCOUNTER_KEY', int_cols=['DIAGNOSIS_CODE'])


**Assigning Diagnosis column for easier filtering**

In [ ]:
bc_df['Diagnosis'] = 'Breast Cancer'

crc_df['Diagnosis'] = 'Colorectal Cancer'

ec_df['Diagnosis'] = 'Endometrial Cancer'

ov_df['Diagnosis'] = 'Ovarian Cancer'

pc_df['Diagnosis'] = 'Pancreatic Cancer'

rc_df['Diagnosis'] = 'Renal Cancer'

In [ ]:
#CONVERT ALL DATE COLUMN TO DATETIME
bc_df['ENCOUNTER_DATE']  = pd.to_datetime(bc_df['ENCOUNTER_DATE'])
crc_df['ENCOUNTER_DATE']  = pd.to_datetime(crc_df['ENCOUNTER_DATE'])
ec_df['ENCOUNTER_DATE']  = pd.to_datetime(ec_df['ENCOUNTER_DATE'])
ov_df['ENCOUNTER_DATE']  = pd.to_datetime(ov_df['ENCOUNTER_DATE'])
pc_df['ENCOUNTER_DATE']  = pd.to_datetime(pc_df['ENCOUNTER_DATE'])
rc_df['ENCOUNTER_DATE']  = pd.to_datetime(rc_df['ENCOUNTER_DATE'])

In [ ]:
#Extract year into its own column
bc_df['year'] = bc_df['ENCOUNTER_DATE'].dt.year
crc_df['year'] = crc_df['ENCOUNTER_DATE'].dt.year
ec_df['year'] = ec_df['ENCOUNTER_DATE'].dt.year
ov_df['year'] = ov_df['ENCOUNTER_DATE'].dt.year
pc_df['year'] = pc_df['ENCOUNTER_DATE'].dt.year
rc_df['year'] = rc_df['ENCOUNTER_DATE'].dt.year

In [ ]:
#Function to count yearly patients
def unique_patients(df):
    return (
        df.drop_duplicates(subset=['PATIENT_IDENTIFIER', 'year']).groupby('year')['PATIENT_IDENTIFIER'].nunique().reindex(range(2017,2022), fill_value=0)
    )

# Criteria Filtering 

**Filter 1: Male With Breast Cancer**

In [ ]:
m_bc = bc_df[bc_df['GENDER'] == 'Male']
m_bc_cnt = m_bc['PATIENT_IDENTIFIER'].nunique()
print(f"There are {m_bc_cnt} Male Breast Cancer Patients")

In [ ]:
f1_counts = unique_patients(m_bc)
f1_counts

In [ ]:
m_bc_pat = m_bc['PATIENT_IDENTIFIER'].unique()
m_bc_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': m_bc_pat})
m_bc_pat_df

**Filter 2: Breast Cancer Diagnosed < 50 years old**

In [ ]:
age_bc = bc_df[bc_df['AGE'] < 50]
age_bc_cnt = age_bc['PATIENT_IDENTIFIER'].nunique()
print(f"There are {age_bc_cnt} Breast Cancer Patients who are less than 50 years old")

In [ ]:
f2_counts = unique_patients(age_bc)
f2_counts

In [ ]:
age_bc_pat = age_bc['PATIENT_IDENTIFIER'].unique()
age_bc_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': age_bc_pat})
age_bc_pat_df

**Filter 3: Colorectal Cancer Diagnosed < 50 years old**

In [ ]:
age_crc = crc_df[crc_df['AGE'] < 50]
age_crc_cnt = age_crc['PATIENT_IDENTIFIER'].nunique()
print(f"There are {age_crc_cnt} Colorectal Cancer Patients who are less than 50 years old")

In [ ]:
f3_counts = unique_patients(age_crc)
f3_counts

In [ ]:
age_crc_pat = age_crc['PATIENT_IDENTIFIER'].unique()
age_crc_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': age_crc_pat})
age_crc_pat_df

**Filter 4: Endometrial (Uterine) Cancer Diagnosed < 50 years old**

In [ ]:
age_ec = ec_df[ec_df['AGE'] < 50]
age_ec_cnt = age_ec['PATIENT_IDENTIFIER'].nunique()
print(f"There are {age_ec_cnt} Endometrial (Uterine) Cancer Patients who are less than 50 years old")

In [ ]:
f4_counts = unique_patients(age_ec)
f4_counts

In [ ]:
age_ec_pat = age_ec['PATIENT_IDENTIFIER'].unique()
age_ec_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': age_ec_pat})
age_ec_pat_df

**Filter 5: Invasive non-mucinous epithelial ovarian, fallopian rube or primary peritoneal cancer**

In [ ]:
#Filter for specific codes was done in SQL
ov_cnt = ov_df['PATIENT_IDENTIFIER'].nunique()
print(f"There are {ov_cnt} Invasive non-mucinous epithelial ovarian, fallopian rube or primary peritoneal cancer patients")

In [ ]:
f4_counts = unique_patients(ov_df)
f4_counts

In [ ]:
ov_df_pat = ov_df['PATIENT_IDENTIFIER'].unique()
ov_df_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': ov_df_pat})
ov_df_pat_df

**Filter 5: Metastatic Pancreatic Cancer dx at any age**

In [ ]:
#Filter  was done for standard pancreatic cancer diagnosis codes, Cannot assess metastases. 
pc_cnt = pc_df['PATIENT_IDENTIFIER'].nunique()
print(f"There are {pc_cnt} patients with pancreatic cancer at any age")

In [ ]:
f5_counts = unique_patients(pc_df)
f5_counts

In [ ]:
pc_df_pat = pc_df['PATIENT_IDENTIFIER'].unique()
pc_df_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': pc_df_pat})
pc_df_pat_df

**Filter 6: Renal cell carcinoma (RCC) < 45 Years old**

In [ ]:
age_rc = rc_df[rc_df['AGE'] < 45]
age_rc_cnt = age_rc['PATIENT_IDENTIFIER'].nunique()
print(f"There are {age_rc_cnt} Renal Cell Carcinoma Patients who are less than 45 years old")

In [ ]:
f6_counts = unique_patients(age_rc)
f6_counts

In [ ]:
age_rc_pat = age_rc['PATIENT_IDENTIFIER'].unique()
age_rc_pat_df = pd.DataFrame({'PATIENT_IDENTIFIER': age_rc_pat})
age_rc_pat_df

In [ ]:
# Flagged patient keys
demo_flagged_df = pd.concat([m_bc_pat_df, age_bc_pat_df, age_crc_pat_df, age_ec_pat_df, ov_df_pat_df, pc_df_pat_df, age_rc_pat_df], ignore_index=True)

In [ ]:
demo_flagged_df.to_csv('demo_flagged_df.csv')